### Link:https://www.tensorflow.org/beta/tutorials/text/nmt_with_attention

In [3]:
import os

import numpy as np
import pandas as pd

import json

from nltk import sent_tokenize

In [4]:
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [5]:
train = pd.read_json("/home/tanmay/Documents/squad 1.1/json_format/stanford-question-answering-dataset/train-v1.1.json")
dev = pd.read_json("/home/tanmay/Documents/squad 1.1/json_format/stanford-question-answering-dataset/dev-v1.1.json")

In [6]:
STANFORD_DATA_PATH = "/home/tanmay/Documents/squad 1.1/json_format/stanford-question-answering-dataset/"

In [7]:
def one_off_corrections(df):
    """
    some specific corrections, simplifications, etc -- plus some manual copy/paste issues make certain
    answers hard to parse

    it looks like answer_start was determined by a blind search for first occurrence of the answer string.
    possible there are systematic issues with this value throughout the dataset that weren't caught by
    the answer validation used here

    TODO: possibly just save / reupload data with these corrections made
    """
    for idx in range(df.shape[0]):
        answer = df.loc[idx, 'text']
        if answer.startswith(". ") or answer.startswith(", "):
            df.loc[idx, 'text'] = answer[2:]
            df.loc[idx, 'answer_start'] += 2

    for _ in range(0, 5):
        df = df.drop(16818 + _)

    df = df.reset_index(drop=True)

    for _ in range(0, 4):
        df = df.drop(38416 + _)

    df = df.reset_index(drop=True)
    # person could not come up with questions and entered junk data
    df = df.drop(38537)

    df = df.reset_index(drop=True)
    df = df.drop(38666)

    df = df.reset_index(drop=True)
    df = df.drop(38693)

    df = df.reset_index(drop=True)

    df.loc[1590, 'answer_start'] = 332
    df.loc[1590, 'text'] = 'seven'

    df.loc[1595, 'answer_start'] = 332
    df.loc[1595, 'text'] = 'seven'

    df.loc[1600, 'answer_start'] = 332
    df.loc[1600, 'text'] = 'seven'

    df.loc[2478, 'answer_start'] = 302
    df.loc[2478, 'text'] = 'three'

    df.loc[2491, 'answer_start'] = 60
    df.loc[
        2491, 'text'] = "the media player included with the iPhone and iPad, a combination of the Music and Videos apps"

    df.loc[2501, 'answer_start'] = 396

    df.loc[3215, 'answer_start'] = 242
    df.loc[3218, 'answer_start'] = 242

    df.loc[3472, 'answer_start'] = 242

    df.loc[3490, 'text'] = "a million"
    df.loc[3490, 'answer_start'] = 268

    df.loc[4051, 'text'] = "over 50"
    df.loc[4051, 'answer_start'] = 327

    df.loc[4168, 'text'] = "92%"
    df.loc[4168, 'answer_start'] = 444

    df.loc[4182, 'answer_start'] = 618

    df.loc[5121, 'text'] = "15 km southwest of Dushanbe"
    df.loc[5121, 'answer_start'] = 499

    df.loc[5919, 'text'] = "seventh"
    df.loc[5919, 'answer_start'] = 983

    df.loc[5924, 'text'] = "seventh"
    df.loc[5924, 'answer_start'] = 983

    df.loc[5990, 'text'] = "three"
    df.loc[5990, 'answer_start'] = 925

    df.loc[5992, 'text'] = "three"
    df.loc[5992, 'answer_start'] = 925

    df.loc[6314, 'text'] = "six"
    df.loc[6314, 'answer_start'] = 292

    df.loc[6316, 'text'] = "six"
    df.loc[6316, 'answer_start'] = 13

    df.loc[6735, 'answer_start'] = 118

    df.loc[6769, 'answer_start'] = 513
    df.loc[6769, 'text'] = "ritual, visualization, physical exercises, and meditation"

    df.loc[6985, 'answer_start'] = 333
    df.loc[6985, 'text'] = "fifteenth"

    df.loc[7528, 'answer_start'] = 39
    df.loc[7528, 'text'] = "fifteenth"

    df.loc[7529, 'answer_start'] = 326
    df.loc[7529, 'text'] = "fifth"

    df.loc[7622, 'answer_start'] = 411
    df.loc[7622, 'text'] = "fifteenth"

    df.loc[7625, 'answer_start'] = 388
    df.loc[7625, 'text'] = "almost a decade, from 2003 to 2012"

    df.loc[7626, 'answer_start'] = 148
    df.loc[7626, 'text'] = "eight"

    df.loc[7676, 'answer_start'] = 87
    df.loc[7676, 'text'] = "nine"

    df.loc[7676, 'answer_start'] = 113
    df.loc[7676, 'text'] = "Director Bruce Gower won a Primetime Emmy Award for Outstanding Directing For A Variety, Music Or Comedy Series in 2009, and the show won a Creative Arts Emmys each in 2007 and 2008, three in 2009, and two in 2011"
    df.loc[7678, 'answer_start'] = 113
    df.loc[7678, 'text'] = "Director Bruce Gower won a Primetime Emmy Award for Outstanding Directing For A Variety, Music Or Comedy Series in 2009, and the show won a Creative Arts Emmys each in 2007 and 2008, three in 2009, and two in 2011"

    df.loc[8189, 'answer_start'] = 310

    df.loc[8735, 'answer_start'] = 137
    df.loc[8735, 'text'] = "its synthesis of and reaction to the world around it"

    df.loc[8835, 'answer_start'] = 373
    df.loc[8835, 'text'] = "300 of which was paid by Cambridge University Press, 200 by the Royal Society of London, and 50 apiece by Whitehead and Russell themselves"

    df.loc[9745, 'answer_start'] = 621

    df.loc[10227, 'answer_start'] = 81

    df.loc[10354, 'answer_start'] = 326
    df.loc[10354, 'text'] = 'one regiment each of artillery, armour, and combat engineers'

    df.loc[11780, 'answer_start'] = 239
    df.loc[11780, 'text'] = '30'

    df.loc[12189, 'answer_start'] = 96
    df.loc[12189, 'text'] = 'two'

    df.loc[12874, 'answer_start'] = 154
    df.loc[12874, 'text'] = 'five'

    df.loc[13060, 'answer_start'] = 272

    df.loc[13232, 'answer_start'] = 505

    df.loc[13787, 'answer_start'] = 362
    df.loc[13787, 'text'] = 'nine'

    df.loc[13791, 'answer_start'] = 189
    df.loc[13791, 'text'] = 'three'

    df.loc[13799, 'answer_start'] = 944
    df.loc[13799, 'text'] = 'four'

    df.loc[13863, 'answer_start'] = 185
    df.loc[13863, 'text'] = 'I, II and III'

    df.loc[14377, 'answer_start'] = 65
    df.loc[14377, 'text'] = 'one'

    df.loc[14448, 'answer_start'] = 78
    df.loc[14448, 'text'] = 'three'

    df.loc[14473, 'answer_start'] = 259
    df.loc[14473, 'text'] = '⟨p⟩'

    df.loc[14516, 'answer_start'] = 341

    df.loc[14582, 'answer_start'] = 12
    df.loc[14582, 'text'] = 'two'

    df.loc[14589, 'answer_start'] = 674
    df.loc[14589, 'text'] = "partial negative"

    df.loc[15159, 'answer_start'] = 108

    df.loc[15246, 'answer_start'] = 186

    df.loc[15865, 'text'] = 'i'
    df.loc[15865, 'answer_start'] = 231

    df.loc[15951, 'text'] = 'one'
    df.loc[15951, 'answer_start'] = 114

    df.loc[16124, 'answer_start'] = 85

    df.loc[16297, 'answer_start'] = 121

    df.loc[17106, 'answer_start'] = 200
    df.loc[17111, 'answer_start'] = 413

    df.loc[17168, 'text'] = 'eight'
    df.loc[17168, 'answer_start'] = 63

    df.loc[17280, 'answer_start'] = 151

    df.loc[18256, 'answer_start'] = 228

    df.loc[18369, 'answer_start'] = 370
    df.loc[18369, 'text'] = 'ser'

    df.loc[18707, 'answer_start'] = 0

    df.loc[23848, 'answer_start'] = 3
    df.loc[23848, 'text'] = 'the 1950s'

    df.loc[25521, 'answer_start'] = 418
    df.loc[25521, 'text'] = 'sync word'
    ##
    df.loc[27972, 'answer_start'] = 111
    df.loc[27972, 'text'] = 'the difficulty of the elements the gymnast attempts and whether or not the gymnast meets composition requirements'
    df.loc[27972, 'question'] = 'How is the start value determined?'

    df.loc[28010, 'answer_start'] = 116
    df.loc[28010, 'text'] = 'external force which the gymnasts have to overcome with their muscle force and has an impact on the gymnasts linear and angular momentum'

    df.loc[34600, 'answer_start'] = 404
    df.loc[34600, 'text'] = 'Men did not show any sexual arousal to non-human visual stimuli'

    df.loc[36071, 'answer_start'] = 222
    df.loc[36071, 'text'] = 'the information must be changed'

    df.loc[39349, 'answer_start'] = 40
    df.loc[39349, 'text'] = 'a US$5 million grant for the International Justice Mission (IJM)'

    df.loc[45018, 'answer_start'] = 454
    df.loc[45018, 'text'] = 'because Florida had become "a derelict open to the occupancy of every enemy, civilized or savage, of the United States, and serving no other earthly purpose than as a post of annoyance to them.'

    df.loc[38536, 'answer_start'] = 491
    df.loc[38536, 'question'] = 'How long did the Rhodians hold out under siege by Demetrius Poliorcetes?'
    df.loc[38536, 'text'] = 'one year'

    df.loc[45076, 'answer_start'] = 562
    df.loc[45076, 'text'] = 'the first post-Reconstruction Republican governor'

    df.loc[45077, 'answer_start'] = 703
    df.loc[45077, 'text'] = "the state's first post-reconstruction Republican US Senator"

    df.loc[47651, 'answer_start'] = 208
    df.loc[47651, 'text'] = 'increasing numbers of airlines have began launching direct flights'

    df.loc[52335, 'answer_start'] = 147
    df.loc[52335, 'text'] = 'Mahmoud Massahi'

    df.loc[53520, 'answer_start'] = 197
    df.loc[53520, 'question'] = 'How many days after the Soviet Union issued their ultimatum did the Romanians meet their demands?'
    df.loc[53520, 'text'] = 'Two'

    df.loc[55205, 'answer_start'] = 48
    df.loc[55205, 'text'] = 'the rebuilt Wembley Stadium'

    df.loc[55752, 'answer_start'] = 370
    df.loc[55752, 'text'] = 'lower-elevation areas of the Piedmont'

    df.loc[56306, 'answer_start'] = 506
    df.loc[56306, 'text'] = 'the wings of flightless birds and the rudiments of pelvis and leg bones found in some snakes'

    df.loc[57712, 'answer_start'] = 78
    df.loc[57712, 'text'] = 'vinyl'

    df.loc[58754, 'answer_start'] = 35
    df.loc[58754, 'text'] = 'the evolving relationship between state governments and the federal government of the United States'

    df.loc[58813, 'answer_start'] = 239
    df.loc[58813, 'question'] = 'How is public spending distributed in Spain?'
    df.loc[58813, 'text'] = 'the central government accounting for just 18% of public spending, 38% for the regional governments, 13% for the local councils, and the remaining 31% for the social security system'

    df.loc[61893, 'answer_start'] = 101
    df.loc[61893, 'text'] = 'Little'

    df.loc[66861, 'answer_start'] = 520
    df.loc[66861, 'text'] = 'parks, schools, public buildings, proper roads and the other amenities that characterise a modern city'

    df.loc[68312, 'answer_start'] = 29
    df.loc[68312, 'question'] = 'What occurs with osmotic diarrhea?'
    df.loc[68312, 'text'] = 'too much water is drawn into the bowels'

    df.loc[70195, 'answer_start'] = 427
    df.loc[70195, 'questions'] = "What did the Native American tribes fail to accomplish in the later Pontiac's War?"
    df.loc[70195, 'text'] = 'returning them to their pre-war status'

    df.loc[70443, 'answer_start'] = 495
    df.loc[70443, 'text'] = 'somehow-belligerent'

    df.loc[71357, 'answer_start'] = 118
    df.loc[71357, 'text'] = 'science-fiction and adventure'

    df.loc[75502, 'text'] = '140 million'
    df.loc[75502, 'answer_start'] = 389

    df.loc[85016, 'answer_start'] = 387

    df.loc[85004, 'question'] = "What was Greece's reference year budget deficit?"
    df.loc[85004, 'text'] = '3.38% of GDP'
    df.loc[85004, 'answer_start'] = 385

    df.loc[85016, 'text'] = "this affected deficit values after 2001 (when Greece had already been admitted into the Eurozone)"
    df.loc[85016, 'answer_start'] = 363

    df.loc[87208, 'text'] = "typically discarded"
    df.loc[87208, 'answer_start'] = 216

    return df

In [8]:
def import_stanford_qa_data():
    # thanks to Bharath, https://www.kaggle.com/bharathsh/stanford-q-a-json-to-clean-dataframe,
    # for code adapted for this use
    #
    train_path = os.path.join(STANFORD_DATA_PATH, 'train-v1.1.json')
    path = ['data', 'paragraphs', 'qas', 'answers']
    with open(train_path) as fh:
        raw_json = json.loads(fh.read())

    js = pd.io.json.json_normalize(raw_json, path)
    m = pd.io.json.json_normalize(raw_json, path[:-1])
    r = pd.io.json.json_normalize(raw_json, path[:-2])

    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx = np.repeat(m['id'].values, m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx

    df = pd.concat([
        m[['id', 'question', 'context']].set_index('id'),
        js.set_index('q_idx')], 1).reset_index()

    df['c_id'] = df['context'].factorize()[0]
    df.head()
    df = one_off_corrections(df)
    return df

In [9]:
train = import_stanford_qa_data()

In [10]:
train.head()

,index,question,context,answer_start,text,c_id,questions
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,NaN
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,NaN
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,NaN
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0,NaN
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0,NaN


In [11]:
train.shape

(87587, 7)

In [12]:
train.shape[0]

87587

In [13]:
train['answer_sent'] = None

In [14]:
train.head()

,index,question,context,answer_start,text,c_id,questions,answer_sent
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,NaN,None
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,NaN,None
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,NaN,None
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0,NaN,None
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0,NaN,None


In [16]:
train['context'].loc[0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [22]:
train[train['context'].str.startswith("Architecturally, the school has a Catholic character")]

,index,question,context,answer_start,text,c_id,questions,answer_sent
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,NaN,None
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,NaN,None
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,NaN,None
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0,NaN,None
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0,NaN,None


In [23]:
train['context'].loc[100]

'One of the main driving forces in the growth of the University was its football team, the Notre Dame Fighting Irish. Knute Rockne became head coach in 1918. Under Rockne, the Irish would post a record of 105 wins, 12 losses, and five ties. During his 13 years the Irish won three national championships, had five undefeated seasons, won the Rose Bowl in 1925, and produced players such as George Gipp and the "Four Horsemen". Knute Rockne has the highest winning percentage (.881) in NCAA Division I/FBS football history. Rockne\'s offenses employed the Notre Dame Box and his defenses ran a 7–2–2 scheme. The last game Rockne coached was on December 14, 1930 when he led a group of Notre Dame all-stars against the New York Giants in New York City.'

In [24]:
train[train['context'].str.startswith("One of the main driving forces in the growth of the")]

,index,question,context,answer_start,text,c_id,questions,answer_sent
98,573387acd058e614000b5cb1,The Notre Dame football team got a new head co...,One of the main driving forces in the growth o...,117,Knute Rockne,20,NaN,None
99,573387acd058e614000b5cb2,What was the amount of wins Knute Rockne attai...,One of the main driving forces in the growth o...,204,105,20,NaN,None
100,573387acd058e614000b5cb5,In what year did the team lead by Knute Rockne...,One of the main driving forces in the growth o...,354,1925,20,NaN,None
101,573387acd058e614000b5cb3,How many years was Knute Rockne head coach at ...,One of the main driving forces in the growth o...,251,13,20,NaN,None
102,573387acd058e614000b5cb4,How many national titles were won when Knute R...,One of the main driving forces in the growth o...,274,three,20,NaN,None


In [98]:
dev.head()

,data,version
0,"{'title': 'Super_Bowl_50', 'paragraphs': [{'co...",1.1
1,"{'title': 'Warsaw', 'paragraphs': [{'context':...",1.1
2,"{'title': 'Normans', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Nikola_Tesla', 'paragraphs': [{'con...",1.1
4,"{'title': 'Computational_complexity_theory', '...",1.1


In [99]:
for row_idx in range(train.shape[0]):
    y = train['text'][row_idx]
    try:
        list_idx = [(y in x) for x in sent_tokenize(train['context'][row_idx])].index(True)
        ans_sent = sent_tokenize(train['context'][row_idx])[list_idx]
        train['answer_sent'][row_idx] = ans_sent
    except:
        train['answer_sent'][row_idx] = None
        

/home/tanmay/python3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tanmay/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
train.head()

,index,question,context,answer_start,text,c_id,questions,answer_sent
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,NaN,"It is a replica of the grotto at Lourdes, Fran..."
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,NaN,Immediately in front of the Main Building and ...
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,NaN,Atop the Main Building's gold dome is a golden...
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0,NaN,"Immediately behind the basilica is the Grotto,..."
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0,NaN,Atop the Main Building's gold dome is a golden...


In [87]:
# train.to_csv("/home/tanmay/Documents/learnogether/data/qna/qna_train.csv", index=False)

In [101]:
train['answer_sent'].isna().sum()

194

train['answer_sent'].isna().sum()
old one contained 216 NaN values..b4 substituting one_off_corrections function

In [59]:
train['text'][0]

'Saint Bernadette Soubirous'